In [2]:
import xarray as xr
import pandas as pd
import rioxarray as rio
from rosetta import rosetta, SoilData
import numpy as np
import os
from p_tqdm import p_map
from tqdm import tqdm
from scipy.interpolate import interp2d


# Set the number of CPUs to use
num_cpus = 48
# Use the taskset command to set the CPU affinity for the current process
os.system('taskset -p -c 0-{} {}'.format(num_cpus-1, os.getpid()))


# turn off deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

pid 1780695's current affinity list: 0-63
pid 1780695's new affinity list: 0-47


/home/jeremyl/.pyenv/versions/3.9.6/envs/venv_SARRA/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Downloading iSDA datasets

In [2]:
# - silt https://zenodo.org/record/4094610
!wget -nc -P ../data/iSDA_data/ https://zenodo.org/record/4094610/files/sol_silt_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif

# - clay https://zenodo.org/record/4085160
!wget -nc -P ../data/iSDA_data/ https://zenodo.org/record/4085160/files/sol_clay_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif

# - sand https://zenodo.org/record/4094607
!wget -nc -P ../data/iSDA_data/ https://zenodo.org/record/4094607/files/sol_sand_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif

# - bulk density https://zenodo.org/record/4087905
!wget -nc -P ../data/iSDA_data/ https://zenodo.org/record/4087905/files/sol_db_od_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif

File ‘../data/iSDA_data/sol_silt_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif’ already there; not retrieving.

File ‘../data/iSDA_data/sol_clay_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif’ already there; not retrieving.

File ‘../data/iSDA_data/sol_sand_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif’ already there; not retrieving.

File ‘../data/iSDA_data/sol_db_od_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif’ already there; not retrieving.



## Defining available water content computation function

In [3]:
def calc_theta_a(args):

    from scipy.interpolate import griddata

    def interpolate_xarray(arr):
        y, x = np.indices(arr.shape)
        points = np.column_stack((y.ravel(), x.ravel()))

        values = arr.values.ravel()
        nan_mask = np.isnan(values)
        missing_points = points[nan_mask]

        if not missing_points.size:
            return arr

        valid_points = points[~nan_mask]
        valid_values = values[~nan_mask]

        interpolated_values = griddata(valid_points, valid_values, missing_points, method='nearest')

        # Reshape the nan_mask to match the shape of the input DataArray
        nan_mask_2d = nan_mask.reshape(arr.shape)

        arr.values[nan_mask_2d] = interpolated_values

        return arr  

    left, bottom, right, top = [args[0], args[1], args[2], args[3]]

    filename = f'theta_a_mm_per_cm_{left}_{bottom}_{right}_{top}.tif'
    filepath = os.path.join('../data/output/', filename)

    if filename in os.listdir('../data/output/'):
        pass
    else:

        data_path = "../data/iSDA_data/"

        path = {
            "sand": os.path.join(data_path, "sol_sand_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif"),
            "silt": os.path.join(data_path, "sol_silt_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif"),
            "clay": os.path.join(data_path, "sol_clay_tot_psa_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif"),
            "bulk_density": os.path.join(data_path, "sol_db_od_m_30m_0..20cm_2001..2017_v0.13_wgs84.tif"),
        }

        # create an empty xarray dataset
        ds = xr.Dataset()

        for layer in path :
            # Open file using xarray
            # da = xr.open_rasterio(path[layer]).squeeze("band", drop=True)
            # open using rioxarray 
            da = rio.open_rasterio(path[layer], masked=True).squeeze("band", drop=True)

            # Subset the DataArray based on the bounding box
            da = da.sel(x=slice(left, right), y=slice(top, bottom))

            # Add the DataArray to the Dataset
            ds[layer] = da

        # prepare rosetta dataframe
        df_rosetta = pd.DataFrame({"sand": ds["sand"].to_numpy().flatten(),
                                "silt": ds["silt"].to_numpy().flatten(),
                                "clay": ds["clay"].to_numpy().flatten(),
                                    "bulk_density": ds["bulk_density"].to_numpy().flatten(),
                                })

        # normalize values
        # df_rosetta["sand_norm"] = df_rosetta["sand"] / df_rosetta[["sand","clay","silt"]].sum(axis=1) * 100.0
        # df_rosetta["silt_norm"] = df_rosetta["silt"] / df_rosetta[["sand","clay","silt"]].sum(axis=1) * 100.0
        # df_rosetta["clay_norm"] = df_rosetta["clay"] / df_rosetta[["sand","clay","silt"]].sum(axis=1) * 100.0
        df_rosetta["bulk_density"] = df_rosetta["bulk_density"] * 10.0 # conversion from raw values to kg/m3
        df_rosetta["bulk_density"] = df_rosetta["bulk_density"] / 1000.0 # conversion from kg/m3 to g/cm3

        
        # compute Rosetta
        # mean, stdev, codes = rosetta(rosetta_version = 3, soildata = SoilData.from_array(df_rosetta[["sand_norm","silt_norm","clay_norm", "bulk_density"]].to_numpy()))
        mean, stdev, codes = rosetta(rosetta_version = 3, soildata = SoilData.from_array(df_rosetta[["sand","silt","clay", "bulk_density"]].to_numpy()))

        
        # reverse flatteing
        ds["theta_r_mean"] = (ds["sand"].dims, mean[:,0].reshape(ds["sand"].shape))
        ds["theta_r_std"] = (ds["sand"].dims, stdev[:,0].reshape(ds["sand"].shape))
        ds["theta_s_mean"] = (ds["sand"].dims, mean[:,1].reshape(ds["sand"].shape))
        ds["theta_s_std"] = (ds["sand"].dims, stdev[:,1].reshape(ds["sand"].shape))
        ds["log10(alpha)"] = (ds["sand"].dims, mean[:,2].reshape(ds["sand"].shape))
        ds["log10(n)"] = (ds["sand"].dims, mean[:,3].reshape(ds["sand"].shape))
        ds["codes"] = (ds["sand"].dims, codes.reshape(ds["sand"].shape))
        # ds["log10(ksat)"] = (ds["sand"].dims, mean[:,4].reshape(ds["sand"].shape))

        # from usda documentation :
        # Column 	Parameter
        # 0 	theta_r, residual water content
        # 1 	theta_s, saturated water content
        # 2 	log10(alpha), 'alpha' shape parameter, log10(1/cm)
        # 3 	log10(npar), 'n' shape parameter
        # 4 	log10(Ksat), saturated hydraulic conductivity, log10(cm/day)

        # formula to compute the volumic water content for a given suction pressure ?
        # the van Genuchten-Mualem model provides a functional relationship between
        # volumetric water content (θ) and soil suction pressure (h). The formula is as
        # follows: θ(h) = θr + [θs - θr] / {1 + (αh)^npar}^(1-1/npar)

        # where:
        #     θr is the residual water content; θs is the saturated water content; α is
        #     the inverse of the characteristic pore size; n is the shape parameter that
        #     governs the degree of curvature of the θ(h) curve; Ksat is the saturated
        #     hydraulic conductivity; h is the soil suction pressure (positive when the
        #     soil is dry and negative when the soil is wet).

        ds["alpha"] = 10.0**ds["log10(alpha)"]
        ds["npar"] = 10.0**ds["log10(n)"]
        # h_fc = 33.0 # kPa
        # h_pwp = 1500.0 # kPa
        h_fc = 33.0 * 10.1972  # kPa to cm H2O
        h_pwp = 1500.0 * 10.1972  # kPa to cm H2O

        # computing the volumetric water content at field capacity and wilting point according to the van Genuchten model
        ds["theta_fc"] = ds["theta_r_mean"] + (ds["theta_s_mean"] - ds["theta_r_mean"]) / (1 + (ds["alpha"] * h_fc) ** ds["npar"])**(1-1/ds["npar"])
        ds["theta_wp"] = ds["theta_r_mean"] + (ds["theta_s_mean"] - ds["theta_r_mean"]) / (1 + (ds["alpha"] * h_pwp) ** ds["npar"])**(1-1/ds["npar"])

        # converting from %vol to mm
        ds["theta_fc_mm"] = ds["theta_fc"] * 10.0 # converting bulk density from raw values to g/cm3, then managing cm/mm conversion
        ds["theta_wp_mm"] = ds["theta_wp"] * 10.0 # converting bulk density from raw values to g/cm3, then managing cm/mm conversion

        # computing the available water content theta_a 
        ds["theta_a"] = ds["theta_fc"] - ds["theta_wp"]
        ds["theta_a_mm"] = ds["theta_fc_mm"] - ds["theta_wp_mm"]  # value in mm of water per vertical cm of soil


        ds["theta_a_mm_interp"] = interpolate_xarray(ds["theta_a_mm"])



        export_list = [
            "theta_r_mean",
            "theta_r_std",
            "theta_s_mean",
            "theta_s_std",
            "alpha",
            "npar",
            "theta_fc",
            "theta_wp",
            "theta_a",
            "theta_a_mm",
            "theta_a_mm_interp",
            "codes",
        ]


        # exporting theta_a_mm as geotiff
        for export_name in export_list:
            
            if not os.path.exists(os.path.join('../data/output/',export_name)):
                os.makedirs(os.path.join('../data/output/',export_name))

            filepath = os.path.join('../data/output/',export_name, f'{export_name}_{left}_{bottom}_{right}_{top}.tif')
            ds[export_name].rio.to_raster(filepath)

        return filename
    

## Define ROI and parameters

In [3]:
# defining dictionary of coordinates for areas of interest
# bounding box coordinates format : [lat NW/top, lon NW/left, lat SE/bottom, lon SE/right]
area = {
    'burkina': [16, -6, 9, 3],
    'burkina_close': [15.1, -5.6, 9.4, 2.4],
    'niger':[23.8, -0.5, 11.3, 15.9],
    'west_africa':[29, -20, 3.5, 26],
    'burkina_bobo_dioulasso': [11.20, -4.3, 11.15, -4.25],
    'burkina_bobo_dioulasso_large': [11.25, -4.3, 11.15, -4.20], # corresponding to [-4.3000, 11.1500, -4.2500, 11.2000] in left, bottom, right, top
    'burkina_bobo_dioulasso_xlarge': [11.4, -4.4, 11.1, -4.10], # corresponding to [-4.3000, 11.1500, -4.2500, 11.2000] in left, bottom, right, top
    'burkina_bobo_dioulasso_xxlarge': [11.6, -4.6, 10.8, -3.90], # corresponding to [-4.3000, 11.1500, -4.2500, 11.2000] in left, bottom, right, top
    }

# selecting area of interest
selected_area = "burkina_close"
resolution = 0.1

## Run

In [5]:
# loading roi coordinates
top, left, bottom, right = [value for value in area[selected_area]]

# Create a list of sub-windows
sub_windows = []
for x in np.arange(left, right, resolution):
    for y in np.arange(bottom, top, resolution):
        sub_windows.append((np.round(x,3), np.round(y,3), np.round(x+resolution,3), np.round(y+resolution,3)))

# parallelizing the computation of calc_theta_a across sub_windows
r = p_map(calc_theta_a, sub_windows, num_cpus=num_cpus, desc="Computing theta_a_mm_0-20cm")

Computing theta_a_mm_0-20cm: 100%|██████████| 16/16 [02:27<00:00,  9.21s/it]


In [4]:
# loading roi coordinates
top, left, bottom, right = [value for value in area[selected_area]]

# Create a list of sub-windows
sub_windows = []
for x in np.arange(left, right, resolution):
    for y in np.arange(bottom, top, resolution):
        sub_windows.append((np.round(x,3), np.round(y,3), np.round(x+resolution,3), np.round(y+resolution,3)))